# Installing CLIP

In [1]:
%pip install ftfy regex tqdm
%pip install git+https://github.com/openai/CLIP.git


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
  Cloning https://github.com/openai/CLIP.git to /private/var/folders/pl/9yyv_2w13q53qr62j77xxb_c0000gn/T/pip-req-build-gtj1_enq
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /private/var/folders/pl/9yyv_2w13q53qr62j77xxb_c0000gn/T/pip-req-build-gtj1_enq
  fatal: unable to access 'https://github.com/openai/CLIP.git/': Could not resolve host: github.com
  error: subprocess-exited-with-error
  
  × git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /private/var/folders/pl/9yyv_2w13q53qr62j77xxb_c0000gn/T/pip-req-build-gtj1_enq did not run successfully.
  │ exit code: 128
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× git clone 

In [8]:
from PIL import Image
import torch
from torch import nn, optim
import glob
import os
import pandas as pd
import json
import numpy as np
import clip
from torch.utils.data import Dataset, DataLoader, BatchSampler
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
import random
from matplotlib.pyplot import imshow
import nltk, re, string, collections
from nltk.util import ngrams
import collections

import sys
sys.path.append('/Users/hanselblanco/Documents/4to/ML/project/bias-project-ML')
from data_preprocess import data_selection

%matplotlib inline

BATCH_SIZE = 4
EPOCH = 25

# Preparing Model and Data

In [9]:
dir_path = '/Users/hanselblanco/Documents/4to/ML/project/bias-project-ML/data/utkface/'
train_df, test_df = data_selection(dir_path)


/Users/hanselblanco/Documents/4to/ML/project/bias-project-ML/data_preprocess.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['gender'][i]= GENDER_MAPPER[df['gender'][i]]
/Users/hanselblanco/Documents/4to/ML/project/bias-project-ML/data_preprocess.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['race'][i]= RACE_MAPPER[df['race'][i]]


## Splitting 20% for Validation

In [10]:
train_df_temp = train_df.sample(frac=0.8)
validation_df = train_df.drop(train_df_temp.index).reset_index(drop=True)
train_df = train_df_temp.reset_index(drop=True)

len(train_df), len(validation_df), len(test_df)

(15173, 3793, 4741)

## Loading Pre-trained CLIP Model and Preprocessor

In [11]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device, jit=False)

## UTKFaceDataset

In [13]:
class UTKFaceDataset(Dataset):
    def __init__(self, dataframe, preprocess):
        self.preprocess = preprocess
        self.filepath = dataframe["filepath"].tolist()
        self.filename = dataframe["filename"].tolist()
        self.gender = dataframe["gender"].tolist()
        self.race = dataframe["race"].tolist()
        self.age = dataframe["age"].tolist()
        self.preprocessed_cache = {}
        for path in self.filepath:
            self.preprocessed_cache[path] = self.preprocess(Image.open(path))

    def __len__(self):
        return len(self.filepath)

    def __getitem__(self, idx):
        filepath = self.filepath[idx]
        filename = self.filename[idx]
        gender = self.gender[idx]
        race = self.race[idx]
        age = self.age[idx]
        image = self.preprocessed_cache[filepath]
        return filepath, filename, gender, race, age, image

train_dataset = UTKFaceDataset(train_df, preprocess)
validation_dataset = UTKFaceDataset(validation_df, preprocess)
len(train_dataset), len(validation_dataset), train_dataset[0]

(15173,
 3793,
 ('/Users/hanselblanco/Documents/4to/ML/project/bias-project-ML/data/utkface/35_0_0_20170117135013511.jpg.chip.jpg',
  '35_0_0_20170117135013511.jpg.chip.jpg',
  'male',
  'white',
  35,
  tensor([[[ 1.3464,  1.1274,  0.8647,  ...,  0.3391,  0.3537,  0.3537],
           [ 1.3318,  1.1274,  0.8501,  ...,  0.2807,  0.2953,  0.2953],
           [ 1.3172,  1.1128,  0.8355,  ...,  0.2223,  0.2369,  0.2515],
           ...,
           [-1.0039, -0.9893, -0.9893,  ...,  1.7260,  1.6092,  1.4194],
           [-1.0039, -1.0039, -0.9893,  ...,  1.6822,  1.5508,  1.3464],
           [-1.0039, -1.0039, -1.0039,  ...,  1.6676,  1.5070,  1.3172]],
  
          [[ 1.2645,  1.0544,  0.7392,  ..., -0.0112,  0.0038,  0.0038],
           [ 1.2495,  1.0393,  0.7242,  ..., -0.0712, -0.0562, -0.0562],
           [ 1.2344,  1.0243,  0.7092,  ..., -0.1463, -0.1313, -0.1163],
           ...,
           [-0.9117, -0.8967, -0.8967,  ...,  1.9398,  1.8047,  1.6547],
           [-0.9117, -0.9117, -0

## BatchSampler

In [15]:
train_dataloader = DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, batch_size = BATCH_SIZE, shuffle=False)

In [21]:
for batch in train_dataloader:
    print(batch)
    break

[('/Users/hanselblanco/Documents/4to/ML/project/bias-project-ML/data/utkface/28_0_4_20170116220531686.jpg.chip.jpg', '/Users/hanselblanco/Documents/4to/ML/project/bias-project-ML/data/utkface/55_0_4_20170117204624770.jpg.chip.jpg', '/Users/hanselblanco/Documents/4to/ML/project/bias-project-ML/data/utkface/22_1_2_20170116165447905.jpg.chip.jpg', '/Users/hanselblanco/Documents/4to/ML/project/bias-project-ML/data/utkface/15_0_0_20170110224250144.jpg.chip.jpg'), ('28_0_4_20170116220531686.jpg.chip.jpg', '55_0_4_20170117204624770.jpg.chip.jpg', '22_1_2_20170116165447905.jpg.chip.jpg', '15_0_0_20170110224250144.jpg.chip.jpg'), ('male', 'male', 'female', 'male'), ('other', 'other', 'asian', 'white'), tensor([28, 55, 22, 15]), tensor([[[[-1.1645, -1.1353, -1.0769,  ..., -0.5222, -0.2010,  0.2369],
          [-1.1645, -1.1353, -1.0915,  ..., -0.6536, -0.2302,  0.3391],
          [-1.1499, -1.1353, -1.1061,  ..., -0.7266, -0.2594,  0.3537],
          ...,
          [-0.4054, -0.3762, -0.3324,  .

# Training

In [22]:
#https://github.com/openai/CLIP/issues/57
def convert_models_to_fp32(model): 
    for p in model.parameters(): 
        p.data = p.data.float() 
        p.grad.data = p.grad.data.float() 

if device == "cpu":
    model.float()

loss_img = nn.CrossEntropyLoss()
loss_txt = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=5e-5,betas=(0.9,0.98),eps=1e-6,weight_decay=0.2)
optimizer = optim.Adam(model.parameters(), lr=1e-5)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, len(train_dataloader)*EPOCH)

In [ ]:
best_te_loss = 1e5
best_ep = -1
for epoch in range(EPOCH):
    print(f"running epoch {epoch}, best test loss {best_te_loss} after epoch {best_ep}")
    step = 0
    tr_loss = 0
    model.train()
    pbar = tqdm(train_dataloader, leave=False)
    for batch in pbar:
        step += 1
        optimizer.zero_grad()

        images, texts, _ = batch
        images = images.to(device)
        texts = clip.tokenize(texts).to(device)
#         print(images.shape, texts.shape)
        logits_per_image, logits_per_text = model(images, texts)
        ground_truth = torch.arange(BATCH_SIZE).to(device)

        total_loss = (loss_img(logits_per_image,ground_truth) + loss_txt(logits_per_text,ground_truth))/2
        total_loss.backward()
        tr_loss += total_loss.item()
        if device == "cpu":
            optimizer.step()
            scheduler.step()
        else:
            convert_models_to_fp32(model)
            optimizer.step()
            scheduler.step()
            clip.model.convert_weights(model)
        pbar.set_description(f"train batchCE: {total_loss.item()}", refresh=True)
    tr_loss /= step
    
    step = 0
    te_loss = 0
    with torch.no_grad():
        model.eval()
        test_pbar = tqdm(test_dataloader, leave=False)
        for batch in test_pbar:
            step += 1
            images, texts, _ = batch
            images = images.to(device)
            texts = clip.tokenize(texts).to(device)
            logits_per_image, logits_per_text = model(images, texts)
            ground_truth = torch.arange(BATCH_SIZE).to(device)

            total_loss = (loss_img(logits_per_image,ground_truth) + loss_txt(logits_per_text,ground_truth))/2
            te_loss += total_loss.item()
            test_pbar.set_description(f"test batchCE: {total_loss.item()}", refresh=True)
        te_loss /= step
        
    if te_loss < best_te_loss:
        best_te_loss = te_loss
        best_ep = epoch
        torch.save(model.state_dict(), "best_model.pt")
    print(f"epoch {epoch}, tr_loss {tr_loss}, te_loss {te_loss}")
torch.save(model.state_dict(), "last_model.pt")

# Evaluating Precision on Validation Set

In [ ]:
model.load_state_dict(torch.load("../input/clipfinetuneweights/best_model.pt"))
NUM_NEG = 127
NUM_TEST = 1000

In [ ]:
n_correct = 0
for i in tqdm(range(NUM_TEST)):
    empty = True
    while empty:
        img_path = random.choice(list(d_test.keys()))
        image = preprocess(Image.open(img_path)).unsqueeze(0).to(device)
        name = img_path.split('/')[-1].split('.')[0]
        caps = d_test[img_path]
        if len(caps) > 0:
            pos_txt = random.choice(caps)
        #         pos_txt = ' '.join(pos_txt)
            empty = False
#     print(pos_txt)
    neg_i = 0
    neg_txts = []
    while neg_i < NUM_NEG:
        img_path = random.choice(list(d_test.keys()))
        neg_name = img_path.split('/')[-1].split('.')[0]
        if neg_name == name:
            continue
        caps = d_test[img_path]
        if len(caps) == 0:
            continue
        neg_txt = random.choice(caps)
        if neg_txt in neg_txts:
            continue
        neg_txts.append(neg_txt)
        neg_i += 1
#     print(name)
#     print(f"Positive caption: {pos_txt}")
#     print(f"Negative caption: {neg_txts}")
    text = clip.tokenize([pos_txt]+neg_txts).to(device)

    with torch.no_grad():
        image_features = model.encode_image(image)
        text_features = model.encode_text(text)

        logits_per_image, logits_per_text = model(image, text)
        probs = logits_per_image.softmax(dim=-1).cpu().numpy()

#     print("Label probs:", probs)
#     print(np.argmax(probs))
    if np.argmax(probs) == 0:
        n_correct +=1
print(f"Test precision {n_correct/NUM_TEST}")

# Evaluating BLEU and Word Diversity using Naive Sampling

## Sampling Captions for Validation Images According to CLIP Text-Image Proximity

In [ ]:
def sample1Caption(img_path, corpus, model, num_cand):
    image = preprocess(Image.open(img_path)).unsqueeze(0).to(device)
    i = 0
    txts = []
    while i < num_cand:
        txt = random.choice(corpus)
        if txt in txts:
            continue
        if len(txt.split())<5 or len(txt)>72:
            continue
        txts.append(txt)
        i += 1
    #     print(name)
    #     print(f"Positive caption: {pos_txt}")
    #     print(f"Negative caption: {neg_txts}")
    text = clip.tokenize(txts).to(device)

    with torch.no_grad():
        logits_per_image, logits_per_text = model(image, text)
        probs = logits_per_image.softmax(dim=-1).cpu().numpy()

    #     print("Label probs:", probs)
    #     print(np.argmax(probs))
    #     imshow(np.asarray(Image.open(img_path)))
    return txts[np.argmax(probs)]

In [ ]:
model.load_state_dict(torch.load("../input/clipfinetuneweights/best_model.pt"))
corpus = []
for txtlist in d_train.values():
    corpus += txtlist
len(corpus), corpus[0]

In [ ]:
captions = {}
for img_path in tqdm(d_test.keys()):
    caption = sample1Caption(img_path, corpus, model, 1000)
    captions[img_path] = caption

## BLEU Score

In [ ]:
for get_bleu in range(1,4):
    bleu_x_lst = []
    bleu_y_lst = []
    for p, caps in d_test.items():
        if not caps:
            continue
        bleu_x_lst.append(captions[p].split())
        splittedcaps = [x.split() for x in caps]
        bleu_y_lst.append(splittedcaps)
    BLEU = torchtext.data.metrics.bleu_score(bleu_x_lst, bleu_y_lst, max_n=get_bleu, weights=[1/get_bleu]*get_bleu)
    print(f"{get_bleu}-gram BLEU score: {BLEU}")

## Word Diversity

In [ ]:
sentences = list(captions.values())
BigramCtr = collections.Counter()
UnigramCtr = collections.Counter()
for sentence in sentences:
    BigramCtr.update(nltk.ngrams(sentence, 2))
    UnigramCtr.update(nltk.ngrams(sentence, 1))
# print("Unigram count:",len(BigramCtr)/len(sentences))
# print("Bigram count:",len(UnigramCtr)/len(sentences))
print("Unigram count:",len(BigramCtr))
print("Bigram count:",len(UnigramCtr))

# Case Analysis on Seen and Unseen Images

In [ ]:
seen_path = random.choice(list(d_train.keys()))
pred_cap_seen = sample1Caption(seen_path, corpus, model, 1000)
gt_cap_seen = d_train[seen_path][:5]
imshow(Image.open(seen_path))
print(f"Some ground truth captions for this seen image: {gt_cap_seen}")
print(f"Caption sampled by fintuned CLIP for this seen image: {pred_cap_seen}")

In [ ]:
unseen_path = random.choice(list(d_test.keys()))
pred_cap_unseen = sample1Caption(unseen_path, corpus, model, 1000)
imshow(Image.open(unseen_path))
gt_cap_unseen = d_test[unseen_path][:5]
print(f"Some ground truth captions for this unseen image: {gt_cap_unseen}")
print(f"Caption sampled by fintuned CLIP for this unseen image: {pred_cap_unseen}")